# Vaja: Državno tekmovanje 2021 - Rešitve

Na naslovu [http://rtk.ijs.si/2021/rtk2021.pdf](http://rtk.ijs.si/2021/rtk2021.pdf) si prenesite pdf z nalogami in jih poskusite rešiti.

Začnite z nalogami iz prve skupine in ko rešite te se lotite nalog druge skupine. Če želite še dodatno preveriti svoje znanje se lotite nalog tretje skupine.

In [12]:
# Prostor za kodo

# Druga skupina: 5. Tetris
import numpy as np

## Testni primer
W, H = 8, 8
plosca = np.zeros((W, H))
seznam_oblik = [
    {
        "oblika": np.ones((2, 2)),
        "st": 6
    },
    {
        "oblika": np.ones((3)),
        "st": 6
    },
    {
        "oblika": np.ones((3, 1)),
        "st": 4
    },
    {
        "oblika": np.array(((0, 1, 0), (1, 1, 1), (0, 1, 0))),
        "st": 2
    },
]

## Podporne funkcije
def StOblik():
    return len(seznam_oblik)

def StPlosckov(oblika):
    return seznam_oblik[oblika]["st"]

def JePokrito(x, y):
    return bool(plosca[y][x])

def PreveriPloscek(oblika, x, y):
    # Get the starting x and y on board
    left_upper = np.transpose(np.nonzero(seznam_oblik[oblika]["oblika"]))[0]
    shape_dim = len(left_upper)
    if shape_dim > 1:
        x -= left_upper[1]
    y -= left_upper[0]

    shape = seznam_oblik[oblika]["oblika"]
    location_on_board = []
    if shape_dim > 1:
        for i, row in enumerate(shape):
            for j, element in enumerate(row):
                x_match = x + j
                y_match = y + i
                # Check for boundry
                if x_match < 0 or x_match >= plosca.shape[1] or y_match < 0 or y_match >= plosca.shape[0]:
                    return False
                # Check for overlap
                if shape[i][j] and plosca[y_match][x_match]:
                    return False
    else:
        for j, element in enumerate(shape):
            x_match = x + j
            y_match = y
            # Check for boundry
            if x_match < 0 or x_match >= plosca.shape[1] or y_match < 0 or y_match >= plosca.shape[0]:
                return False
            # Check for overlap
            if shape[j] and plosca[y_match][x_match]:
                return False

    return True
            
def PostaviPloscek(oblika, x, y, b):
    # Check if any of the shapes remaining
    if b and not seznam_oblik[oblika]["st"]:
        raise ValueError(f"Ran out of shape {oblika}.")
    # Get the starting x and y on board
    left_upper = np.transpose(np.nonzero(seznam_oblik[oblika]["oblika"]))[0]
    shape_dim = len(left_upper)
    if shape_dim > 1:
        x -= left_upper[1]
    y -= left_upper[0]

    shape_id = 1 * (oblika + 1)
    shape = seznam_oblik[oblika]["oblika"].copy()
    shape *= shape_id
    location_on_board = []
    if shape_dim > 1:
        for i, row in enumerate(shape):
            for j, element in enumerate(row):
                x_match = x + j
                y_match = y + i
                # Check for boundry
                if x_match < 0 or x_match >= plosca.shape[1] or y_match < 0 or y_match >= plosca.shape[0]:
                    raise ValueError(f"Placement/removal was out of bounds at x = {x_match}, y = {y_match}.")
                # Check for overlap
                if b and shape[i][j] and plosca[y_match][x_match]:
                    raise ValueError(f"Placement failed. No space for shape at x = {x_match}, y = {y_match}.")

                if not b and shape[i][j] and plosca[y_match][x_match] != shape_id:
                    raise ValueError(f"Removal failed. This shape is not present at location x = {x_match}, y = {y_match}.")

                # If shape write the location on board
                if shape[i][j]:
                    location_on_board.append((y_match, x_match))
    else:
        for j, element in enumerate(shape):
            x_match = x + j
            y_match = y
            # Check for boundry
            if x_match < 0 or x_match >= plosca.shape[1] or y_match < 0 or y_match >= plosca.shape[0]:
                raise ValueError(f"Placement/removal was out of bounds at x = {x_match}, y = {y_match}.")
            # Check for overlap
            if b and shape[j] and plosca[y_match][x_match]:
                raise ValueError(f"Placement failed. No space for shape at x = {x_match}, y = {y_match}.")

            if not b and shape[j] and plosca[y_match][x_match] != shape_id:
                raise ValueError(f"Removal failed. This shape is not present at location x = {x_match}, y = {y_match}.")
                
            # If shape write the location on board
            if shape[j]:
                location_on_board.append((y_match, x_match))

    
    if b:
        # Add the shape to the board
        plosca[tuple(zip(*location_on_board))] = shape_id
        seznam_oblik[oblika]["st"] -= 1
    else:
        # Remove the shape from the board
        plosca[tuple(zip(*location_on_board))] = 0
        seznam_oblik[oblika]["st"] += 1


## Rešitev ##

# Funkcija vrne True, če je uspela v celoti pokriti ploščo.
# Sicer vrne False, plošča pa je ob vrnitvi iz funkcije v enakem stanju
# kot na začetku klica.
def NadaljujPokrivanje(x, y, prosti):
    # Poiščimo naslednje nepokrito polje.
    while y < H and JePokrito(x, y):
        x += 1
        if x == W:
            x = 0
            y += 1
    if y >= H:
        return True # Če je vse že pokrito, smo končali.
    # Na vse možne načine ga poskusimo pokriti
    for oblika, oblika_prosti in enumerate(prosti):
        # Ali lahko sem postavimo plošček te oblike?
        if not oblika_prosti:
            continue
        if not PreveriPloscek(oblika, x, y):
            continue
        
        # Postavimo ga in nadaljujmo z rekurzijo.
        PostaviPloscek(oblika, x, y, True)
        prosti[oblika] -= 1
        if NadaljujPokrivanje(x, y, prosti):
            return True
        # Če nismo uspeli pokriti cele plošče, plošček spet odstranimo.
        PostaviPloscek(oblika, x, y, False)
        prosti[oblika] += 1
    return False # Če pridemo do sem, se plošče ni dalo pokriti do konca.

def PokrijPlosco():
    n = StOblik()
    prosti = [oblika["st"] for oblika in seznam_oblik]
    return NadaljujPokrivanje(0, 0, prosti)

print(PokrijPlosco())
print(plosca)


True
[[2. 2. 2. 1. 1. 1. 1. 3.]
 [3. 1. 1. 1. 1. 1. 1. 3.]
 [3. 1. 1. 2. 2. 2. 4. 3.]
 [3. 2. 2. 2. 4. 4. 4. 4.]
 [2. 2. 2. 4. 4. 4. 4. 3.]
 [3. 2. 2. 2. 4. 1. 1. 3.]
 [3. 1. 1. 1. 1. 1. 1. 3.]
 [3. 1. 1. 1. 1. 2. 2. 2.]]
